In [192]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import pickle

In [207]:
해외종목정보 = pd.read_csv("NH_CONTEST_NW_FC_STK_IEM_IFO.csv", encoding="EUC-KR", skipinitialspace=True)
종목일자별시세 = pd.read_csv("NH_CONTEST_STK_DT_QUT.csv", encoding="EUC-KR", skipinitialspace=True)
주식일별정보 = pd.read_csv("NH_CONTEST_NHDATA_STK_DD_IFO.csv", encoding="EUC-KR", skipinitialspace=True)
매수매도계좌정보 = pd.read_csv("NH_CONTEST_NHDATA_IFW_OFW_IFO.csv", encoding="EUC-KR", skipinitialspace=True)
고객보유정보 = pd.read_csv("NH_CONTEST_NHDATA_CUS_TP_IFO.csv", encoding="EUC-KR", skipinitialspace=True)
ETF구성종목정보 = pd.read_csv("NH_CONTEST_DATA_ETF_HOLDINGS.csv", encoding="EUC-KR", skipinitialspace=True)
ETF배당내역 = pd.read_csv("NH_CONTEST_DATA_HISTORICAL_DIVIDEND.csv", encoding="EUC-KR", skipinitialspace=True)
ETF점수정보 = pd.read_csv("NH_CONTEST_ETF_SOR_IFO.csv", encoding="EUC-KR", skipinitialspace=True)

data_name = ["해외종목정보", "종목일자별시세", "주식일별정보", "매수매도계좌정보", "고객보유정보", "ETF구성종목정보", "ETF배당내역", "ETF점수정보"]

In [210]:
# 변수명 변경 + 공백제거
해외종목정보.columns = ["티커종목코드", "외화증권한글명", "외화증권영문명", "주식/ETF구분코드",
                   "상장주식총수량", "외화시장구분코드", "회사주소", "웹주소", "업종분류명",
                   "CEO명", "영문사업개요내용", "섹터분류명", "산업명", "시가총액"]
해외종목정보 = 해외종목정보.map(lambda x: x.strip() if isinstance(x, str) else x)

종목일자별시세.columns = ["거래일자", "티커종목코드", "종목시가", "종목고가", "종목저가",
                   "종목종가", "전일대비증감가격", "전일대비증감율", "누적거래수량",
                   "거래대금", "매도체결합계수량", "매수체결합계수량", "환율"]
종목일자별시세 = 종목일자별시세.map(lambda x: x.strip() if isinstance(x, str) else x)

주식일별정보.columns = ["거래일자", "티커종목코드", "총보유계좌수", "총보유수량",
                   "당사평균보유수량", "당사평균보유비중비율", "당사평균평가손익",
                   "당사평균매입단가", "당사평균수익율", "당사평균보유기간일수",
                   "분포상위10퍼센트수치", "분포상위30퍼센트수치", "분포상위50퍼센트수치",
                   "분포상위70퍼센트수치", "분포상위90퍼센트수치", "기준종가", "손실투자자비율",
                   "수익투자자비율", "신규매수계좌수", "전량매도계좌수", "종목조회건수",
                   "관심종목등록건수"]
주식일별정보 = 주식일별정보.map(lambda x: x.strip() if isinstance(x, str) else x)

매수매도계좌정보.columns = ["기준일자", "티커종목코드", "유입/유출구분코드", "유입유출티커코드",
                    "유입유출금액비중", "유입유출랭크"]
매수매도계좌정보 = 매수매도계좌정보.map(lambda x: x.strip() if isinstance(x, str) else x)

고객보유정보.columns = ["기준일자", "티커종목코드", "고객구성대분류코드", "고객구성중분류코드",
                  "고객구성계좌수비율", "고객구성투자비율"]
고객보유정보 = 고객보유정보.map(lambda x: x.strip() if isinstance(x, str) else x)

ETF구성종목정보.columns = ["대상ETF티커", "ETF개별구성종목티커", "보유종목의가치(USD)", "보유종목의영문명", "보유종목의한글명",
                     "보유종목의주수(주)", "보유종목의비중(%)", "보유종목의타입(ST:주식,EF:ETF,EN:ETN,SSEF:Single-StockETF)"]
ETF구성종목정보 = ETF구성종목정보.map(lambda x: x.strip() if isinstance(x, str) else x)

ETF배당내역.columns = ["대상ETF티커", "배당락일(YYYYMMDD)", "배당금", "수정배당금", "배당기준일(YYYYMMDD)",
                   "지급일(YYYYMMDD)", "공시일(YYYYMMDD)", "배당주기(Quarterly:분기배당,Weekly:주배당,Monthly:월배당,SemiAnnual:반기배당,Annual:연배당,Other:알수없음)"]
ETF배당내역 = ETF배당내역.map(lambda x: x.strip() if isinstance(x, str) else x)

ETF점수정보.columns = ["거래일자", "대상ETF티커", "1개월총수익율", "3개월총수익율",
                   "1년총수익율", "ETF점수", "ETFZ점수", "Z점수순위",
                   "누적수익율Z점수", "정보비율Z점수", "샤프지수Z점수", "상관관계Z점수",
                   "트래킹에러Z점수", "최대낙폭Z점수", "변동성Z점수"]
ETF점수정보 = ETF점수정보.map(lambda x: x.strip() if isinstance(x, str) else x)

In [211]:
고객보유정보["티커종목코드"].unique()

array(['AAL', 'AAOI', 'AAON', ..., 'ZM', 'ZNTL', 'ZS'], dtype=object)

In [ ]:
# 거래일자, 고객 중분류 검색하면 딕셔너리 만드는 코드
# 여기에는 함수만 올림
def filter_date(df, date):
    if '기준일자' in df.columns:
        return df[df['기준일자'] == date]
    elif '거래일자' in df.columns:
        return df[df['거래일자'] == date]

def filter_code(df, ticker_code):
    if '티커종목코드' in df.columns:
        return df[df['티커종목코드'] == ticker_code]
    elif '대상ETF티커' in df.columns:
        return df[df['대상ETF티커'] == ticker_code]

def filter_bigConstraint(df, big_constraint):
    return df[df['고객구성대분류코드'] == big_constraint]

# 날짜, 고객 별로 ETF를 얼마나 갖고 있는지 저장할 dictionary
# 이름을 뭐로 해야할지 모르겠어서 name_mijung ㅋㅋ
name_mijung = {}

def add_or_concat(key, new_data):
    # 데이터프레임 생성 (new_data는 새로운 row을 담은 DataFrame)
    new_df = pd.DataFrame(new_data)

    # 해당 key에 데이터프레임이 없는 경우, 새로운 데이터프레임 추가
    if key not in name_mijung:
        name_mijung[key] = new_df
    else:
        # 이미 데이터프레임이 있다면, pd.concat으로 row 추가
        name_mijung[key] = pd.concat([name_mijung[key], new_df])

def calculate_actual_numbers(고객보유정보, 주식일별정보, date, ticker_code, big_constraint):
    filtered_stock_info = filter_date((filter_code(주식일별정보, ticker_code)), date)
    rowIndex_stock_info = filtered_stock_info.index[0]
    filtered_customer_info = filter_bigConstraint(filter_date((filter_code(고객보유정보, ticker_code)), date), big_constraint)
    for rowIndex in filtered_customer_info.index:
        accounts = filtered_customer_info.loc[rowIndex, "고객구성계좌수비율"] * filtered_stock_info.loc[rowIndex_stock_info, "총보유계좌수"]
        stocks = filtered_customer_info.loc[rowIndex, "고객구성투자비율"] * filtered_stock_info.loc[rowIndex_stock_info, "총보유수량"]
        dont_know_proper_name = pd.DataFrame({"보유고객계좌수":[accounts], "보유고객주식수":[stocks]}, index=[ticker_code])
        add_or_concat((date, filtered_customer_info.loc[rowIndex, "고객구성중분류코드"]), dont_know_proper_name)

In [ ]:
#이거 돌리면 위에서 저장한 name_mijung이라는 딕셔너리가 만들어짐
tickerCode = list(주식일별정보['티커종목코드'].unique())
for tC in tickerCode:
# tC = tickerCode[189]
    imsi = 주식일별정보[주식일별정보['티커종목코드'] == tC]
    ETF_dates = list(imsi['거래일자'].unique())
    for date in ETF_dates:
        for i in range(1,4): #고객 대분류 번호가 1, 2, 3
            calculate_actual_numbers(고객보유정보, 주식일별정보, date, tC, i)
            print(f"{date}, {tC}, 대분류 :{i}")

# ETF 하나 당 9.8sec, 우리가 가진 ETF 1277개
# ETA : 3시간 42분 
# 우리 멋쟁이 코드 화이팅 !

# 파일 저장
# folder_path 세팅해주세요
file_path = f"{folder_path}날짜및고객분류별보유ETF.pkl"
with open(file_path, 'wb') as f:
    pickle.dump(name_mijung, f)